In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/Notebooks/Luodingo

import torch 
import torch.optim as optim

import seq2seq_multilayer_gru
from sequence_model_trainer import TrainModel

from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset
from torchtext.data import Iterator, BucketIterator

%load_ext autoreload
%autoreload 2

/content/gdrive/My Drive/Colab Notebooks/Notebooks/Luodingo
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
MASKED_TEXT = Field(
                sequential=True,
                tokenize=lambda x: x.split(), 
             )

TARGET_TEXT = Field(
                sequential=True,
                tokenize=lambda x: x.split(), 
             )

# LABEL = LabelField(dtype=torch.long)

fields = [('id', None), ('keywords', MASKED_TEXT), ('target', TARGET_TEXT)]

In [16]:
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/Datasets
train, val, test = TabularDataset.splits(
                            path='Masked Corpus',
                            train='train.csv',
                            validation='val.csv',
                            test='test.csv',
                            format='csv',
                            skip_header=True,
                            fields=fields
                    )

/content/gdrive/My Drive/Colab Notebooks/Datasets


In [0]:
MASKED_TEXT.build_vocab(train)
TARGET_TEXT.build_vocab(train)
# LABEL.build_vocab(train)

In [0]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iter, val_iter, test_iter = BucketIterator.splits(
                            (train, val, test),
                            batch_size=BATCH_SIZE,
                            device=device,
                            sort_key=lambda x: len(x.keywords))

In [0]:
# EMB_DIM=bert.config.to_dict()['hidden_size']
EMB_DIM=256
ENC_INPUT_DIM=len(MASKED_TEXT.vocab)
DEC_INPUT_DIM=len(TARGET_TEXT.vocab)
OUTPUT_DIM=DEC_INPUT_DIM
N_LAYER=4
HID_DIM=1024
DROPOUT=0.3

model = seq2seq_multilayer_gru.Seq2Seq(
              enc_input_dim=ENC_INPUT_DIM,
              dec_input_dim=DEC_INPUT_DIM,
              emb_dim=EMB_DIM,
              enc_hid_dim=HID_DIM,
              dec_hid_dim=HID_DIM,
              n_layers=N_LAYER,
              output_dim=OUTPUT_DIM,    
              device=device,
              dropout=DROPOUT
         ).to(device)

In [0]:
LEARNING_RATE = 0.0001

adam = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
cross_e = torch.nn.CrossEntropyLoss()

In [0]:
trainer = TrainModel(
            model=model,
            train_iterator=train_iter,
            val_iterator=val_iter,
            optimizer=adam,
            criterion=cross_e,
            output_dim=OUTPUT_DIM
          )

In [0]:
N_EPOCHS = 200
CLIP = 1
trainer.epoch(n_epochs=N_EPOCHS, clip=CLIP, model_name='seq2seq-multilayer-gru.pt')

Epoch: 01 | Time: 0m 51s
	Train Loss: 3.570 | Train PPL:  35.501
	 Val. Loss: 5.155 |  Val. PPL: 173.218
Epoch: 02 | Time: 0m 51s
	Train Loss: 3.024 | Train PPL:  20.564
	 Val. Loss: 5.115 |  Val. PPL: 166.445
Epoch: 03 | Time: 0m 51s
	Train Loss: 2.951 | Train PPL:  19.126
	 Val. Loss: 5.139 |  Val. PPL: 170.565
Epoch: 04 | Time: 0m 51s
	Train Loss: 2.892 | Train PPL:  18.024
	 Val. Loss: 5.233 |  Val. PPL: 187.324
Epoch: 05 | Time: 0m 51s
	Train Loss: 2.821 | Train PPL:  16.801
	 Val. Loss: 5.093 |  Val. PPL: 162.935
Epoch: 06 | Time: 0m 51s
	Train Loss: 2.740 | Train PPL:  15.490
	 Val. Loss: 4.900 |  Val. PPL: 134.298
Epoch: 07 | Time: 0m 51s
	Train Loss: 2.694 | Train PPL:  14.797
	 Val. Loss: 4.945 |  Val. PPL: 140.430
Epoch: 08 | Time: 0m 51s
	Train Loss: 2.714 | Train PPL:  15.091
	 Val. Loss: 4.972 |  Val. PPL: 144.382
Epoch: 09 | Time: 0m 51s
	Train Loss: 2.646 | Train PPL:  14.092
	 Val. Loss: 4.867 |  Val. PPL: 129.919
Epoch: 10 | Time: 0m 51s
	Train Loss: 2.534 | Train PPL